# Set up

In [ ]:
import io
import pprint
import requests
import json
from keycloak import KeycloakAdmin
from keycloak import KeycloakOpenID
from IPython.display import Image

APP_BASE_URL='http://localhost:8082'
APP_ADMIN_USERNAME='admin@everest.engineering'
APP_ADMIN_PASSWORD='ac0n3x72'

KEYCLOAK_BASE_URL='http://localhost:8180'
KEYCLOAK_ADMIN_USERNAME='admin@everest.engineering'
KEYCLOAK_ADMIN_PASSWORD='ac0n3x72'

# This needs to be generated in Keycloak for the 'default-client' in the 'default' realm. See the top level README.md
OID_CLIENT_SECRET='7nX8oIKYH7xXNXbDJ0oiz5byC3aqrMrC'

pp = pprint.PrettyPrinter()

oid_client = KeycloakOpenID(
    server_url=f"{KEYCLOAK_BASE_URL}/auth/",
    realm_name="default",
    client_id="default-client",
    client_secret_key=OID_CLIENT_SECRET,
    verify=False)

## Application admin login

In [ ]:
admin_tokens = oid_client.token(APP_ADMIN_USERNAME, APP_ADMIN_PASSWORD)
pp.pprint(admin_tokens)

In [ ]:
admin_userinfo = oid_client.userinfo(admin_tokens['access_token'])
pp.pprint(admin_userinfo)

# System info

## OID configuration

In [ ]:
pp.pprint(oid_client.well_know())

## Application version

In [ ]:
print(requests.get(f'{APP_BASE_URL}/api/version').text)

## Health status

In [ ]:
public_health = requests.get(f'{APP_BASE_URL}/actuator/health')
pp.pprint(public_health.json())

In [ ]:
private_health = requests.get(
    f'{APP_BASE_URL}/actuator/health', 
    headers=
    {
        "Authorization": f"Bearer {admin_tokens['access_token']}"
    }
)
print(private_health.status_code)
pp.pprint(private_health.json())

## Available system metrics

Most metrics won't show up until later due ot lazy loading.

In [ ]:
metrics = requests.get(
    f'{APP_BASE_URL}/actuator/metrics', 
    headers=
    {
        "Authorization": f"Bearer {admin_tokens['access_token']}"
    }
)
print(metrics.status_code)
pp.pprint(metrics.json()['names'])

## HTTP server request metrics

In [ ]:
request_metrics = requests.get(
    f'{APP_BASE_URL}/actuator/metrics/http.server.requests', 
    headers=
    {
        "Authorization": f"Bearer {admin_tokens['access_token']}"
    }
)
print(metrics.status_code)
pp.pprint(request_metrics.json())

## Prometheus metrics

Most metrics won't show up until later due ot lazy loading.

In [ ]:
prometheus_metrics = requests.get(
    f'{APP_BASE_URL}/actuator/prometheus', 
    headers=
    {
        "Authorization": f"Bearer {admin_tokens['access_token']}"
    }
)
print(prometheus_metrics.status_code)
pp.pprint(prometheus_metrics.text)

## Replay status

In [ ]:
replay_status = requests.get(
    f'{APP_BASE_URL}/actuator/replay', 
    headers=
    {
        "Authorization": f"Bearer {admin_tokens['access_token']}"
    }
)
print(replay_status.status_code)
pp.pprint(replay_status.json())

## Trigger replay

Replays when multiple nodes are involved require you to first shut down the tracking event processors so that ownership is released. The node that receives the request to replay will be the one handling it.

This can be done via the dashboard or via `axonserver-cli`.

In [ ]:
response = requests.post(
    f'{APP_BASE_URL}/actuator/replay', 
    headers=
    {
        "Authorization": f"Bearer {admin_tokens['access_token']}"
    },
    json={}
)
# You are expecting a 204 NO-CONTENT response here
print(response.status_code)

# Self registration scenario

## Simulated user self registration for Bob

In [ ]:
keycloak_admin = KeycloakAdmin(
    server_url=f"{KEYCLOAK_BASE_URL}/auth/",
    username=KEYCLOAK_ADMIN_USERNAME,
    password=KEYCLOAK_ADMIN_PASSWORD,
    realm_name='default',
    user_realm_name='master',
    verify=False)

In [ ]:
bob_user_id = keycloak_admin.create_user(
    {
        "email": "bob@example.com",
        "username": "bob@example.com",
        "enabled": True,
        "firstName": "Bob",
        "lastName": "Example",
        "credentials": [{"value": "password-here", "type": "password"}]
    }
)
pp.pprint(bob_user_id)

## Bob's user and organisation does not exist until first use

In [ ]:
response = requests.get(f'{APP_BASE_URL}/api/users',
    headers=
    {
        "Authorization": f"Bearer {admin_tokens['access_token']}"
    }
)
pp.pprint(response.json())

In [ ]:
response = requests.get(f'{APP_BASE_URL}/admin/organizations',
    headers=
    {
        "Authorization": f"Bearer {admin_tokens['access_token']}"
    }
)
pp.pprint(response.json())

## Bob logs in

In [ ]:
bob_tokens = oid_client.token('bob@example.com', 'password-here')
pp.pprint(bob_tokens)

## Bob retrieves profile details

After this call you can go back up and check - the user and their default organisation should now be available

In [ ]:
response = requests.get(f'{APP_BASE_URL}/api/user',
    headers=
    {
        "Authorization": f"Bearer {bob_tokens['access_token']}"
    }
).json()
pp.pprint(response)
bob_org_id = response['organizationId']

## Bob uploads profile photo

In [ ]:
with open('profile.jpg', 'rb') as image:
    response = requests.post(f'{APP_BASE_URL}/api/user/profile-photo',
        headers=
        {
            "Authorization": f"Bearer {bob_tokens['access_token']}"
        },
        files= 
        {
            'file': image
        }
    )
    print(response.status_code)

## Bob downloads profile photo

In [ ]:
response = requests.get(f'{APP_BASE_URL}/api/user/profile-photo/thumbnail?width=1024&height=512',
    headers=
    {
        "Authorization": f"Bearer {bob_tokens['access_token']}"
    }
)
print(response.status_code)
Image(data=response.content)

# Bob does stuff

## Bob lists users on his organisation

In [ ]:
response = requests.get(f'{APP_BASE_URL}/api/organizations/{bob_org_id}/users',
    headers=
    {
        "Authorization": f"Bearer {bob_tokens['access_token']}"
    }
)
print(response.status_code)
pp.pprint(response.json())

## Bob retrieves information about the organisation

In [ ]:
response = requests.get(f'{APP_BASE_URL}/api/organizations/{bob_org_id}',
    headers=
    {
        "Authorization": f"Bearer {bob_tokens['access_token']}"
    }
)
print(response.status_code)
pp.pprint(response.json())

## Bob starts listening for organisation updates

Content negotiation allows you to subscribe to streaming updates. 

This isn't easily demonstrated in this notebook. Run the following to generate the `curl` command that lets you subscribe to streaming updates:

In [ ]:
print(f"curl --verbose -H \"Accept: application/x-ndjson\" -H \"Authorization: Bearer {bob_tokens['access_token']}\" {APP_BASE_URL}/api/organizations/{bob_org_id}")

## Bob changes the organisation's website

In [ ]:
response = requests.put(f'{APP_BASE_URL}/api/organizations/{bob_org_id}',
    headers=
    {
        "Authorization": f"Bearer {bob_tokens['access_token']}"
    },
    json = 
    {
        'websiteUrl': 'https://my.nice.example.com/1'
    }
)
print(response.status_code)

## Bob creates Alice, a new user for the organisation

This relies on the realm being configured to require email verification for the new user to be able to log in.

The application will trigger the verification email as part of the set up process.

For this to work you need to configure your realm in Keycloak to:
* send emails via an SMTP host
* require email verfication

(Don't forget to change Alice's email address)

In [ ]:
response = requests.post(f'{APP_BASE_URL}/api/organizations/{bob_org_id}/users',
    headers=
    {
        "Authorization": f"Bearer {bob_tokens['access_token']}"
    },
    json={
        "displayName": "Alice Example",
        "username": "alice@example.com"
    }
)
print(response.status_code)
print(response.text)
alice_user_id = response.text

## Alice logs into Bob's organisation

In [ ]:
alice_tokens = oid_client.token('alice@example.com', 'password-is-set-via-keycloak-email')
pp.pprint(alice_tokens)

# Admin deletes user and discards encryption key

After this call you can go back up and check that replays still work and that Bob's information no longer appears anywhere in the API or event log. 

Note that the access token will still be valid until it expires. You can turn on (very costly) token validation on every request if necessary.

In [ ]:
response = requests.post(f'{APP_BASE_URL}/api/users/{bob_user_id}/forget',
    headers=
    {
        'Authorization': f"Bearer {admin_tokens['access_token']}"
    },
    json={
      'requestReason': 'GDPR request received'
    }                  
)
print(response)